# First MSSP example

In [ ]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

## Manage datas

Read the synthetic instance of Section 3.2

In [ ]:
synthetic_5x5_df = pd.read_csv("data/d_it_ij_5x5_1it.csv",
                               index_col=[0, 1],
                               decimal=",")

# start conting nodes from 0 makes life easier 
synthetic_5x5_df.index = [synthetic_5x5_df.index.get_level_values(0) - 1,
                          synthetic_5x5_df.index.get_level_values(1) - 1]

synthetic_5x5_df.head()

it1
0 5  3.0
  6  1.0
1 5  1.0
  6  1.0
  7  1.0

In [ ]:
class WArc:

    def __init__(self, begin, end, weight, index):
        self.i = begin
        self.j = end
        self.w = weight
        self.idx = index

    def __repr__(self):
        return f"{self.__class__.__name__}({self.i!r}, {self.j!r}, {self.w!r}, {self.idx!r})"



class Agent:
    
    def __init__(self, name, source, terminus, index):
        self.name = name
        self.source = source
        self.terminus = terminus
        self.idx = index
        self.path = None

    def __repr__(self):
        return f"{self.__class__.__name__}({self.name!r}, {self.source!r}, {self.terminus!r}, {self.idx!r})"

Set up datas that will be used to solve the problem

In [ ]:
def get_nodes(networks_df, fishing_from):

    if fishing_from == "cols":
        starting_nodes = networks_df.columns.get_level_values(0)
        ending_nodes = networks_df.columns.get_level_values(1)
    elif fishing_from == "indx":
        starting_nodes = networks_df.index.get_level_values(0)
        ending_nodes = networks_df.index.get_level_values(1)

    return [i for i in starting_nodes.union(ending_nodes).unique()]

In [ ]:
w_arcs = [WArc(*arc, weight, idx) for idx, (arc, weight) in enumerate(synthetic_5x5_df.itertuples())]

nodes = get_nodes(synthetic_5x5_df,
                  fishing_from="indx")

agents_names = [0, 1, 2, 3]
agents_sources = [0, 2, 3, 4]
agents_terminus = [20, 22, 23, 24]
agents = [Agent(name, s, t, idx) for idx, (name, s, t) in enumerate(zip(agents_names, agents_sources, agents_terminus))]

## Manage the problem

Create the problem

In [ ]:
MSPP_pb = gb.Model("First MSPP")
MSPP_pb.setParam("OutputFlag", 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10


Define decision variables

In [ ]:
var_shape = len(w_arcs), len(agents)

X = MSPP_pb.addMVar(var_shape,
                    vtype=GRB.BINARY,  # 5) Binary constraints
                    name="X")


Define the objective function

In [ ]:
# 1-3) Objective function
MSPP_pb.setObjective(
    gb.quicksum(
        arc.w * X[arc.idx, agent.idx]
        for arc in w_arcs for agent in agents
    ),
    GRB.MINIMIZE
)

Add constraints

In [ ]:
# 4) Flow constraints

def compute_flow(X, node, w_arcs, agent):
    flow_out = gb.quicksum(
        X[arc.idx, agent.idx]
        for arc in w_arcs if arc.i == node
    )
    flow_in = gb.quicksum(
        X[arc.idx, agent.idx]
        for arc in w_arcs if arc.j == node
    )
    return flow_out - flow_in


for agent in agents:
    for node in nodes:
        if node == agent.source:
            MSPP_pb.addConstr(compute_flow(X, node, w_arcs, agent) == 1)
        elif node == agent.terminus:
            MSPP_pb.addConstr(compute_flow(X, node, w_arcs, agent) == -1)
        else:
            MSPP_pb.addConstr(compute_flow(X, node, w_arcs, agent) == 0)


Solve the problem

In [ ]:
MSPP_pb.optimize()

Report results

In [ ]:
print("Result of the optimization is:")
if MSPP_pb.Status == 2:
    print("optimal")
elif MSPP_pb.Status == 3:
    print("infeasible")
elif MSPP_pb.Status == 5:
    print("unbounded")
else:
    print("Some other return status")

Result of the optimization is:
optimal


In [ ]:
print(f"Minimum distance covered is {MSPP_pb.ObjVal}")

Minimum distance covered is 16.0


In [ ]:
for agent in agents:
    print(f"Agent {agent.name} will follow the path:")
    for arc in w_arcs:
        if math.isclose(X.x[arc.idx, agent.idx], 1):
            print(f"{arc.i}->{arc.j}", end="\t")
    print()

#TODO: plot the paths

Agent 0 will follow the path:
0->6	6->12	12->16	16->20	
Agent 1 will follow the path:
2->7	7->12	12->17	17->22	
Agent 2 will follow the path:
3->7	7->12	12->17	17->23	
Agent 3 will follow the path:
4->8	8->14	14->18	18->24	
